In [11]:
import torch
from torch.utils.data import Dataset
from torchvision.transforms import Compose

import glob
from typing import Any
import pandas as pd



In [14]:
class CustomDataset(Dataset):
    def __init__(self,pathname,transform=None,target_transform=None):
        self.file_names= glob.glob(pathname+ '\*\*.txt')
        self.class_map= {'S':0,'Z':1}
        self.transform= transform
        self.target_transform= target_transform
        
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self,indx):
        filename= self.file_names[indx]
        df= pd.read_csv(filename,header=None)
        chr= filename.split('\\')[-1][0]
        
        sample= df.iloc[:,0].to_numpy()
        label= self.class_map[chr]
        if self.transform:
            sample= self.transform(sample)
        if self.target_transform:
            label= self.target_transform(label)   
        return sample,label

class ToTensor:
    def __call__(self, sample):
        sample = torch.from_numpy(sample).type(torch.float32)
        return sample
    
class Normalize:
    def __init__(self, mu, std):
        self.mu = mu
        self.std = std
    def __call__(self,sample):
        sample = (sample-self.mu) / self.std
        return sample

In [17]:
pathname='EEG'
to_tensor = ToTensor()
to_normalize = Normalize(1,100)
transfrom= Compose([to_tensor,to_normalize])
ds= CustomDataset(pathname,transfrom)

sample,label=ds[2]